## Preparing your dataset
You should create a diverse set of demonstration conversations that are similar to the conversations you will ask the model to respond to at inference time in production.
- The training dataset must include examples of the cases that the model failed to give you the desired answer and make sure to add the suitable answer in the training dataset
- Dataset can be created manually or downloaded from online websites like (HuggingFace)[https://huggingface.co/]
  - Example: this is a fine tuning dataset on huggingface https://huggingface.co/datasets/Hyder12/Fine-tuning-gpt-3.5-Dataset?row=13
- To fine-tune a model, you are required to provide at least 10 examples. We typically see clear improvements from fine-tuning on 50 to 100 training examples with gpt-3.5-turbo but the right number varies greatly based on the exact use case.
  - Start with 50 well-crafted demonstrations and seeing if the model shows signs of improvement after fine-tuning.

In [ ]:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

In [3]:
import json

data = []
with open("assets/Fine-Tuning_Data.jsonl", "r", encoding='utf-8') as file:
    for line in file:
        # loading each line as a JSON object and appending it to the data list
        data.append(json.loads(line))


# creating the dataset in the ChatCompletion API format
ds_formatted = [
    {"messages": [
        {"role": "system", "content": "You are a smart assistant answer the clients questions about our products."},
        {'role': 'user', 'content': x["question"]},
        {'role': 'assistant', 'content': x["answer"]}]} for x in data
]


# checking the format
for line in ds_formatted:
    print(line)


# creating the datasets for training and evaluation
ds_train = ds_formatted[:25]
ds_eval = ds_formatted[25:50]


# saving the training dataset to train.jsonl
with open('train.jsonl', 'w') as f:
    for line in ds_train:
        json.dump(line, f)
        f.write('\n')

# saving the evaluation dataset to val.jsonl
with open('eval.jsonl', 'w') as f:
    for line in ds_eval:
        json.dump(line, f)
        f.write('\n')

{'messages': [{'role': 'system', 'content': 'You are a smart assistant answer the clients questions about our products.'}, {'role': 'user', 'content': 'What types of furniture does ComfortLiving offer?'}, {'role': 'assistant', 'content': 'ComfortLiving offers a wide range of furniture including sofas, beds, dining tables, chairs, and storage solutions.'}]}
{'messages': [{'role': 'system', 'content': 'You are a smart assistant answer the clients questions about our products.'}, {'role': 'user', 'content': 'Where can I buy ComfortLiving furniture?'}, {'role': 'assistant', 'content': 'You can purchase ComfortLiving furniture through our online store or at any of our physical retail locations.'}]}
{'messages': [{'role': 'system', 'content': 'You are a smart assistant answer the clients questions about our products.'}, {'role': 'user', 'content': 'Does ComfortLiving offer home delivery?'}, {'role': 'assistant', 'content': 'Yes, ComfortLiving offers home delivery services for all purchases m

## Upload Data
- You need a paid plan to test the fine tuning
  

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
# find_dotenv() find .env automatically by walking up directories until it's found 
# load_dotenv() load the environment variables from the .env file
# override=True allows the .env file to override the system environment variables
load_dotenv(find_dotenv(), override=True)

apiKey = os.environ.get('OPENAI_API_KEY')

In [6]:
from openai import OpenAI

client = OpenAI(api_key=apiKey)

In [7]:
train_data = client.files.create(
  file=open("train.jsonl", "rb"),
  purpose="fine-tune"
)
train_data

FileObject(id='file-M8a5iHgOnyvA7f8OUeG92tg3', bytes=8610, created_at=1717421919, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [8]:
val_data = client.files.create(
  file=open("eval.jsonl", "rb"),
  purpose="fine-tune"
)
val_data

FileObject(id='file-iS1RY0oFJIThPAQnBKVFZiSR', bytes=8912, created_at=1717421993, filename='eval.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [9]:
print(train_data.id)
print(val_data.id)

file-M8a5iHgOnyvA7f8OUeG92tg3
file-iS1RY0oFJIThPAQnBKVFZiSR


## Create Fine Tuning Job
- The job will take some time so you can check for the status every while
- Once the job is done you will receive an Email with the id of the job and how you can test your model

In [10]:
job = client.fine_tuning.jobs.create(
    validation_file=val_data.id,
    model="gpt-3.5-turbo",
    training_file=train_data.id)
print(job)

FineTuningJob(id='ftjob-qTWtVwL1zTKwquo1hSzjdXTY', created_at=1717422257, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-XDHCAu3meW0XfegcmqqsSq1s', result_files=[], status='validating_files', trained_tokens=None, training_file='file-M8a5iHgOnyvA7f8OUeG92tg3', validation_file='file-iS1RY0oFJIThPAQnBKVFZiSR', user_provided_suffix=None, seed=1653693330, estimated_finish=None, integrations=[])


In [22]:
# Retrieve the state of a fine-tune
status = client.fine_tuning.jobs.retrieve(job.id)
print(status.status)

succeeded


In [21]:
#list all fine-tuning jobs
jobs = client.fine_tuning.jobs.list()
print(jobs)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-qTWtVwL1zTKwquo1hSzjdXTY', created_at=1717422257, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9W2MJQeA', finished_at=1717422588, hyperparameters=Hyperparameters(n_epochs=4, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-XDHCAu3meW0XfegcmqqsSq1s', result_files=['file-rq6NrcyuihIIWFPKVbSoZxbk'], status='succeeded', trained_tokens=5316, training_file='file-M8a5iHgOnyvA7f8OUeG92tg3', validation_file='file-iS1RY0oFJIThPAQnBKVFZiSR', user_provided_suffix=None, seed=1653693330, estimated_finish=None, integrations=[])], object='list', has_more=False)


In [24]:
events = client.fine_tuning.jobs.list_events(job.id)


In [25]:
events_list = events.data
events_list.reverse()

for event in events_list:
    print(event.message)

Step 85/100: training loss=0.10, validation loss=0.71
Step 86/100: training loss=0.00, validation loss=0.49
Step 87/100: training loss=0.02, validation loss=1.08
Step 88/100: training loss=0.05, validation loss=1.10
Step 89/100: training loss=0.01, validation loss=0.43
Step 90/100: training loss=0.10, validation loss=0.21
Step 91/100: training loss=0.01, validation loss=1.09
Step 92/100: training loss=0.01, validation loss=0.41
Step 93/100: training loss=0.14, validation loss=0.51
Step 94/100: training loss=0.00, validation loss=1.29
Step 95/100: training loss=0.17, validation loss=1.10
Step 96/100: training loss=0.06, validation loss=0.99
Step 97/100: training loss=0.01, validation loss=1.57
Step 98/100: training loss=0.07, validation loss=1.71
Step 99/100: training loss=0.00, validation loss=0.46
Step 100/100: training loss=0.07, validation loss=0.77, full validation loss=0.76
Checkpoint created at step 50 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9W2MJJQk:ckpt-step-50
Checkp

## Using the new Model

In [41]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a smart assistant answer the clients questions about our products."},
        {"role": "user", "content": "my cat damaged my new couch that i bought from ComfortLiving. so how you will fix it?"}
    ]
)

print(response.choices[0].message.content)

I'm sorry to hear about the damage to your couch. Please refer to the warranty and return policy provided by ComfortLiving for specific details on how to address this issue. It is a good idea to reach out to their customer service team for assistance in finding a solution to repair or replace the damaged couch.


In [28]:
print(job.model)

gpt-3.5-turbo-0125


In [45]:
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal::9W2MJQeA",
    messages=[
        {"role": "system", "content": "You are a smart assistant answer the clients questions about ComfortLiving products."},
        #{"role": "user", "content": "What is the warranty that you support?"}
        #{"role": "user", "content": "what is the cancellation policy?"}
        {"role": "user", "content": "my cat damaged my new couch that i bought from ComfortLiving. so how you will fix it?"}
    ]
)

print(response.choices[0].message.content)

I'm sorry to hear about the damage to your couch. ComfortLiving offers a warranty on furniture for manufacturing defects, but damages caused by pets are not covered.
